In [0]:
# =============================================================================
# TRAVEL BOOKING SCD2 MERGE PROJECT - SILVER LAYER: CUSTOMER DIMENSION SCD2
# =============================================================================
# This notebook implements SCD2 (Slowly Changing Dimension Type 2) for customer data
# Purpose: Maintains historical versions of customer attributes with temporal tracking
# SCD2 Logic: Closes old versions and creates new versions when attributes change
# Output: Creates/updates customer_dim table with surrogate keys and temporal columns

from delta.tables import DeltaTable
from pyspark.sql import functions as F

# =============================================================================
# PARAMETER EXTRACTION WITH DEFAULTS
# =============================================================================
# Extract widget parameters with fallback defaults for flexibility
# arrival_date: Business date for processing (defaults to today)
# catalog: Unity Catalog name (defaults to travel_bookings)
# schema: Target schema (defaults to default)

import datetime as _dt
try:
    arrival_date = dbutils.widgets.get("arrival_date")
except Exception:
    arrival_date = _dt.date.today().strftime("%Y-%m-%d")
try:
    catalog = dbutils.widgets.get("catalog")
except Exception:
    catalog = "travel_bookings"
try:
    schema = dbutils.widgets.get("schema")
except Exception:
    schema = "default"

# =============================================================================
# SOURCE DATA PREPARATION
# =============================================================================
# Load customer data from bronze layer for the specified business date
# Filters to current day's data for incremental processing

src = spark.table(f"{catalog}.bronze.customer_inc").where(F.col("business_date") == F.to_date(F.lit(arrival_date)))

dim_full_name = f"{catalog}.{schema}.customer_dim"

# =============================================================================
# DIMENSION TABLE SCHEMA DEFINITION
# =============================================================================
# Create customer dimension table with surrogate key and SCD2 columns
# customer_sk: Surrogate key (auto-generated identity)
# customer_id: Natural key from source system
# SCD2 columns: valid_from, valid_to, is_current for temporal tracking

spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{schema}")
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {dim_full_name} (
  customer_sk BIGINT GENERATED ALWAYS AS IDENTITY,
  customer_id INT,
  customer_name STRING,
  customer_address STRING,
  email STRING,
  valid_from DATE,
  valid_to DATE,
  is_current BOOLEAN
) USING DELTA
""")

# =============================================================================
# SCD2 MERGE LOGIC
# =============================================================================
# Implement SCD2 pattern: close old versions and create new versions
# Step 1: Close current rows where attributes have changed
# Step 2: Insert new current versions for changed rows

if spark.catalog.tableExists(dim_full_name):
  dim = DeltaTable.forName(spark, dim_full_name)

  # Step 1: Close current rows where attributes change
  # Updates valid_to and is_current for existing records with changed attributes
  (dim.alias("d").merge(
      src.alias("s"),
      "d.customer_id = s.customer_id AND d.is_current = true"
    )
    .whenMatchedUpdate(
      condition="d.customer_name <> s.customer_name OR d.customer_address <> s.customer_address OR d.email <> s.email",
      set={"valid_to": "s.valid_from", "is_current": "false"}
    )
    .whenNotMatchedInsert(values={
      "customer_id": "s.customer_id",
      "customer_name": "s.customer_name",
      "customer_address": "s.customer_address",
      "email": "s.email",
      "valid_from": "s.valid_from",
      "valid_to": "s.valid_to",
      "is_current": "true"
    })
    .execute())

  # Step 2: Insert new current versions for changed rows
  # Creates new records for customers with changed attributes
  # Surrogate key (customer_sk) is auto-generated by IDENTITY column
  changed = spark.sql(f"""
    SELECT s.customer_id, s.customer_name, s.customer_address, s.email, s.valid_from, s.valid_to, true AS is_current
    FROM {catalog}.bronze.customer_inc s
    LEFT JOIN {dim_full_name} d
      ON d.customer_id = s.customer_id AND d.valid_from = s.valid_from AND d.is_current = true
    WHERE s.business_date = DATE('{arrival_date}') AND d.customer_id IS NULL
  """)
  if changed.count() > 0:
    changed.write.mode("append").format("delta").saveAsTable(dim_full_name)
else:
  # =============================================================================
  # INITIAL LOAD
  # =============================================================================
  # For first-time setup, load all customer data as current versions
  # Surrogate keys will be auto-generated during initial load
  
  init = src.select("customer_id","customer_name","customer_address","email","valid_from","valid_to","is_current")
  init.write.mode("append").format("delta").saveAsTable(dim_full_name)

print("SCD2 (dimension) merge complete")
